In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [2]:
def onehotencoder(df, cols):
    res_df = pd.DataFrame()
    for col in cols:
        for feature in df[col].unique():
            res_df[col+'_' +feature] = (df[col]==feature).astype('int')
    return res_df

In [3]:
pd.options.display.max_rows = 500

## Appliation Train

In [4]:
# reading and selecting 100000
original_app = pd.read_csv('data/application_train.csv')
app_df = original_app.sample(100000, random_state=42)
app_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
245895,384575,0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
98194,214010,0,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
36463,142232,0,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
249923,389171,0,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
158389,283617,0,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0


## Bureau and Bureau Balance

In [10]:
# reading bureau
bur_df = pd.read_csv('data/bureau.csv')
bur_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [11]:
bur_text = onehotencoder(bur_df[['CREDIT_TYPE']],['CREDIT_TYPE'])

In [12]:
bur_df['CREDIT_TYPE'].value_counts()

Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin lending)          4
Mobile operator loan                                  1
Interbank credit                                      1
Name: CREDIT_TYPE, dtype: int64

In [13]:
# reading bureau balance 
bur_balance_df = pd.read_csv('data/bureau_balance.csv')
bur_balance_df.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [14]:
# selecting the newest record in bureau and getting the total length of records in month
temp = bur_balance_df.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].idxmax()
temp = bur_balance_df.iloc[list(temp.values)]
temp = temp.merge(
    pd.DataFrame(bur_balance_df.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].count()).reset_index().rename(columns={'MONTHS_BALANCE':'BALANCE_LENGTH'}), 
    on = 'SK_ID_BUREAU')
temp.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,BALANCE_LENGTH
0,5001709,0,C,97
1,5001710,0,C,83
2,5001711,0,X,4
3,5001712,0,C,19
4,5001713,0,X,22


In [21]:
# merging the bureau and bureau balance 
temp_1 = bur_df.merge(temp, on="SK_ID_BUREAU", suffixes=('_bur', '_bal'),how='left')
temp_1.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE,STATUS,BALANCE_LENGTH
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN,NaN


In [22]:
temp_1['active'] = temp_1['CREDIT_ACTIVE'] == 'Active'

In [23]:
temp_1['STATUS'] = temp_1['STATUS'].fillna('X')

In [24]:
bur_text = onehotencoder(temp_1, ['CREDIT_CURRENCY','CREDIT_TYPE','STATUS'])
temp_1 = temp_1.merge(bur_text,left_index=True,right_index=True)
temp_1.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for purchase of shares (margin lending),STATUS_X,STATUS_C,STATUS_0,STATUS_1,STATUS_2,STATUS_5,STATUS_4,STATUS_3
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,0,0,1,0,0,0,0,0,0,0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,0,0,1,0,0,0,0,0,0,0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,0,0,1,0,0,0,0,0,0,0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,0,0,1,0,0,0,0,0,0,0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,...,0,0,1,0,0,0,0,0,0,0


In [25]:
active_prop =temp_1.groupby('SK_ID_CURR')['active'].sum() /temp_1.groupby('SK_ID_CURR')['CREDIT_ACTIVE'].count()
bureau_cond = pd.DataFrame(active_prop,columns=['ACTIVE_PROP']).reset_index()
bureau_cond.head()

,SK_ID_CURR,ACTIVE_PROP
0,100001,0.428571
1,100002,0.250000
2,100003,0.250000
3,100004,0.000000
4,100005,0.666667


In [26]:
temp_1 = temp_1.drop(columns=['SK_ID_BUREAU','CREDIT_ACTIVE','CREDIT_CURRENCY','CREDIT_TYPE','STATUS','active'])

In [27]:
temp_agg =temp_1.groupby('SK_ID_CURR').aggregate(['max', 'sum']).reset_index()
temp_agg.columns= temp_agg.columns.map('_'.join)
temp_agg.head()

,SK_ID_CURR_,DAYS_CREDIT_max,DAYS_CREDIT_sum,CREDIT_DAY_OVERDUE_max,CREDIT_DAY_OVERDUE_sum,DAYS_CREDIT_ENDDATE_max,DAYS_CREDIT_ENDDATE_sum,DAYS_ENDDATE_FACT_max,DAYS_ENDDATE_FACT_sum,AMT_CREDIT_MAX_OVERDUE_max,...,STATUS_1_max,STATUS_1_sum,STATUS_2_max,STATUS_2_sum,STATUS_5_max,STATUS_5_sum,STATUS_4_max,STATUS_4_sum,STATUS_3_max,STATUS_3_sum
0,100001,-49,-5145,0,0,1778.0,577.0,-544.0,-3302.0,NaN,...,1,1,0,0,0,0,0,0,0,0
1,100002,-103,-6992,0,0,780.0,-2094.0,-36.0,-4185.0,5043.645,...,0,0,0,0,0,0,0,0,0,0
2,100003,-606,-5603,0,0,1216.0,-2178.0,-540.0,-3292.0,0.000,...,0,0,0,0,0,0,0,0,0,0
3,100004,-408,-1734,0,0,-382.0,-977.0,-382.0,-1065.0,0.000,...,0,0,0,0,0,0,0,0,0,0
4,100005,-62,-572,0,0,1324.0,1318.0,-123.0,-123.0,0.000,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# merging both bur and bur_balance with current application 
app_df = app_df.merge(temp_agg,left_on='SK_ID_CURR',right_on='SK_ID_CURR_',how ='left',suffixes=('_app', '_bur'))
app_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,STATUS_1_max,STATUS_1_sum,STATUS_2_max,STATUS_2_sum,STATUS_5_max,STATUS_5_sum,STATUS_4_max,STATUS_4_sum,STATUS_3_max,STATUS_3_sum
0,384575,0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,214010,0,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,142232,0,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,389171,0,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,283617,0,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# saving to csv
# app_df.to_csv('app_df_bur.csv',index=False)

In [30]:
app_df.shape

(100000, 205)

## Previous Application

In [31]:
# reading previous application
prev_app = pd.read_csv('data/previous_application.csv')
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# getting interesting numeric and text data, text data filling
# fill na in AMT_DOWN_PAYMENT
prev_app_temp = prev_app[['SK_ID_PREV','SK_ID_CURR','AMT_ANNUITY','AMT_APPLICATION','AMT_CREDIT', 'AMT_DOWN_PAYMENT',
                          'NFLAG_INSURED_ON_APPROVAL', 'CNT_PAYMENT', 'DAYS_TERMINATION']]
prev_app_text = prev_app[['NAME_CASH_LOAN_PURPOSE','NAME_CONTRACT_STATUS','NAME_PAYMENT_TYPE','NAME_CLIENT_TYPE',
                          'NAME_SELLER_INDUSTRY','NAME_GOODS_CATEGORY', 'NAME_CONTRACT_TYPE',
                          'PRODUCT_COMBINATION']].fillna('XNA').replace('XAP','XNA')


In [34]:
prev_app_temp['MISSING_DOWN_PAYMENT'] = prev_app_temp['AMT_DOWN_PAYMENT'].isna().astype('int')
prev_app_temp.head()

/var/folders/w1/vc3f0wy917q9_qnbxvz9gvfh0000gn/T/ipykernel_37797/2426437043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_app_temp['MISSING_DOWN_PAYMENT'] = prev_app_temp['AMT_DOWN_PAYMENT'].isna().astype('int')


,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,NFLAG_INSURED_ON_APPROVAL,CNT_PAYMENT,DAYS_TERMINATION,MISSING_DOWN_PAYMENT
0,2030495,271877,1730.430,17145.0,17145.0,0.0,0.0,12.0,-37.0,0
1,2802425,108129,25188.615,607500.0,679671.0,NaN,1.0,36.0,365243.0,1
2,2523466,122040,15060.735,112500.0,136444.5,NaN,1.0,12.0,365243.0,1
3,2819243,176158,47041.335,450000.0,470790.0,NaN,1.0,12.0,-177.0,1
4,1784265,202054,31924.395,337500.0,404055.0,NaN,NaN,24.0,NaN,1


In [35]:
# merging prev useful numeric data + one hot encoding
prev_app_temp=prev_app_temp.merge(onehotencoder(prev_app_text,list(prev_app_text.columns)), 
                                  left_index=True, right_index=True)
prev_app_temp.head(5)

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,NFLAG_INSURED_ON_APPROVAL,CNT_PAYMENT,DAYS_TERMINATION,MISSING_DOWN_PAYMENT,...,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_Cash Street: low,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_Cash Street: middle,PRODUCT_COMBINATION_POS others without interest,PRODUCT_COMBINATION_XNA
0,2030495,271877,1730.430,17145.0,17145.0,0.0,0.0,12.0,-37.0,0,...,0,0,0,0,0,0,0,0,0,0
1,2802425,108129,25188.615,607500.0,679671.0,NaN,1.0,36.0,365243.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2523466,122040,15060.735,112500.0,136444.5,NaN,1.0,12.0,365243.0,1,...,0,0,0,0,0,0,0,0,0,0
3,2819243,176158,47041.335,450000.0,470790.0,NaN,1.0,12.0,-177.0,1,...,0,0,0,0,0,0,0,0,0,0
4,1784265,202054,31924.395,337500.0,404055.0,NaN,NaN,24.0,NaN,1,...,0,0,0,0,0,0,0,0,0,0


In [36]:
prev_app_temp = prev_app_temp.fillna(0)

## POS CASH Balance

In [37]:
# reading pos cash balance
pos = pd.read_csv('POS_CASH_balance.csv')
pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [38]:
pos.shape

(10001358, 8)

In [39]:
# one hot encode name contract status and fill with XNA 
pos_text = onehotencoder(pos[['NAME_CONTRACT_STATUS']].fillna('XNA'),['NAME_CONTRACT_STATUS'])
pos_text[['SK_ID_CURR','SK_ID_PREV']] = pos[['SK_ID_CURR','SK_ID_PREV']]
pos_text.head()

,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_XNA,NAME_CONTRACT_STATUS_Amortized debt,SK_ID_CURR,SK_ID_PREV
0,1,0,0,0,0,0,0,0,0,182943,1803195
1,1,0,0,0,0,0,0,0,0,367990,1715348
2,1,0,0,0,0,0,0,0,0,397406,1784872
3,1,0,0,0,0,0,0,0,0,269225,1903291
4,1,0,0,0,0,0,0,0,0,334279,2341044


In [40]:
# adding new features to pos df 
mod_pos = pos.copy()
mod_pos['CNT_INSTALMENT_PAST'] = pos['CNT_INSTALMENT'] - pos['CNT_INSTALMENT_FUTURE']
mod_pos['INSTALMENT_PROP'] = mod_pos['CNT_INSTALMENT_PAST']/mod_pos['CNT_INSTALMENT']
mod_pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,CNT_INSTALMENT_PAST,INSTALMENT_PROP
0,1803195,182943,-31,48.0,45.0,Active,0,0,3.0,0.062500
1,1715348,367990,-33,36.0,35.0,Active,0,0,1.0,0.027778
2,1784872,397406,-32,12.0,9.0,Active,0,0,3.0,0.250000
3,1903291,269225,-35,48.0,42.0,Active,0,0,6.0,0.125000
4,2341044,334279,-35,36.0,35.0,Active,0,0,1.0,0.027778


In [41]:
# taking the most recent pos record for each account and constructing variable acc length representing how long the account existed
pos_recent = mod_pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmin()
pos_recent = mod_pos.iloc[pos_recent].set_index('SK_ID_PREV')
pos_recent['ACC_LENGTH'] = mod_pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].min() - mod_pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].max()
pos_recent.head()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,CNT_INSTALMENT_PAST,INSTALMENT_PROP,ACC_LENGTH
SK_ID_PREV,,,,,,,,,,
1000001,158271,-10,12.0,12.0,Active,0,0,0.0,0.0,-2
1000002,101962,-54,4.0,4.0,Active,0,0,0.0,0.0,-4
1000003,252457,-4,12.0,12.0,Active,0,0,0.0,0.0,-3
1000004,260094,-29,10.0,10.0,Active,0,0,0.0,0.0,-7
1000005,176456,-56,10.0,10.0,Active,0,0,0.0,0.0,-10


In [42]:
# getting the one hot encoding data of the newest record status 
pos_text = onehotencoder(pos_recent[['NAME_CONTRACT_STATUS']].fillna('XNA'),['NAME_CONTRACT_STATUS'])
pos_text[['SK_ID_CURR']] = pos_recent[['SK_ID_CURR']]
pos_text = pos_text.groupby('SK_ID_CURR').sum()
pos_text.head()

,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_XNA
SK_ID_CURR,,,,,,,
100001,2,0,0,0,0,0,0
100002,1,0,0,0,0,0,0
100003,3,0,0,0,0,0,0
100004,1,0,0,0,0,0,0
100005,0,0,1,0,0,0,0


In [43]:
# dropping the text data 
pos_recent = pos_recent.drop(columns=['NAME_CONTRACT_STATUS']).reset_index(drop=True)
pos_recent.head()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,CNT_INSTALMENT_PAST,INSTALMENT_PROP,ACC_LENGTH
0,158271,-10,12.0,12.0,0,0,0.0,0.0,-2
1,101962,-54,4.0,4.0,0,0,0.0,0.0,-4
2,252457,-4,12.0,12.0,0,0,0.0,0.0,-3
3,260094,-29,10.0,10.0,0,0,0.0,0.0,-7
4,176456,-56,10.0,10.0,0,0,0.0,0.0,-10


In [44]:
# merging to get the mean for each current applicant 
pos_merge = pos_recent.groupby('SK_ID_CURR').mean()
pos_merge['PREVIOUS_POS_ACC'] = pos_recent.groupby('SK_ID_CURR')['MONTHS_BALANCE'].count()
pos_merge = pos_merge.drop(columns='MONTHS_BALANCE')
pos_merge = pos_merge.merge(pos_text,left_index=True, right_index=True).fillna(0)
pos_merge.head()

,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,CNT_INSTALMENT_PAST,INSTALMENT_PROP,ACC_LENGTH,PREVIOUS_POS_ACC,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_XNA
SK_ID_CURR,,,,,,,,,,,,,,,
100001,4.0,3.0,0.0,0.0,1.0,0.25,-3.500000,2,2,0,0,0,0,0,0
100002,24.0,24.0,0.0,0.0,0.0,0.00,-18.000000,1,1,0,0,0,0,0,0
100003,10.0,10.0,0.0,0.0,0.0,0.00,-8.333333,3,3,0,0,0,0,0,0
100004,4.0,4.0,0.0,0.0,0.0,0.00,-3.000000,1,1,0,0,0,0,0,0
100005,0.0,0.0,0.0,0.0,0.0,0.00,-10.000000,1,0,0,1,0,0,0,0


In [45]:
app_df = app_df.merge(pos_merge,how='left',left_on='SK_ID_CURR',right_index=True,suffixes=('','_pos'))
app_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INSTALMENT_PROP,ACC_LENGTH,PREVIOUS_POS_ACC,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_XNA
0,384575,0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,...,0.0,-10.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,214010,0,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,...,0.0,-4.5,8.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0
2,142232,0,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,...,0.0,-9.5,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,389171,0,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,...,0.0,-24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,283617,0,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,...,0.0,-5.5,4.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0


## Credit Card Balance

In [46]:
# reading credit
credit = pd.read_csv('data/credit_card_balance.csv')
credit.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [47]:
# one hot encoding the name contract status
credit_text = onehotencoder(credit,['NAME_CONTRACT_STATUS'])
credit_text.head()

,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Approved
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0


In [48]:
[list(credit.columns)[3:15]]

[['AMT_BALANCE',
  'AMT_CREDIT_LIMIT_ACTUAL',
  'AMT_DRAWINGS_ATM_CURRENT',
  'AMT_DRAWINGS_CURRENT',
  'AMT_DRAWINGS_OTHER_CURRENT',
  'AMT_DRAWINGS_POS_CURRENT',
  'AMT_INST_MIN_REGULARITY',
  'AMT_PAYMENT_CURRENT',
  'AMT_PAYMENT_TOTAL_CURRENT',
  'AMT_RECEIVABLE_PRINCIPAL',
  'AMT_RECIVABLE',
  'AMT_TOTAL_RECEIVABLE']]

In [49]:
list(credit.columns)[3:15] + ['SK_DPD','SK_DPD_DEF']

['AMT_BALANCE',
 'AMT_CREDIT_LIMIT_ACTUAL',
 'AMT_DRAWINGS_ATM_CURRENT',
 'AMT_DRAWINGS_CURRENT',
 'AMT_DRAWINGS_OTHER_CURRENT',
 'AMT_DRAWINGS_POS_CURRENT',
 'AMT_INST_MIN_REGULARITY',
 'AMT_PAYMENT_CURRENT',
 'AMT_PAYMENT_TOTAL_CURRENT',
 'AMT_RECEIVABLE_PRINCIPAL',
 'AMT_RECIVABLE',
 'AMT_TOTAL_RECEIVABLE',
 'SK_DPD',
 'SK_DPD_DEF']

In [50]:
# taking the mean of some variables
credit_1 = credit.groupby(['SK_ID_CURR','SK_ID_PREV'])[list(credit.columns)[3:15] + ['SK_DPD','SK_DPD_DEF']].mean()
credit_1.head()

,,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,SK_DPD,SK_DPD_DEF
SK_ID_CURR,SK_ID_PREV,,,,,,,,,,,,,,
100006,1489396,0.000000,270000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100011,1843384,54482.111149,164189.189189,2432.432432,2432.432432,0.0,0.0,3956.221849,4843.064189,4520.067568,52402.088919,54433.179122,54433.179122,0.000000,0.000000
100013,2038692,18159.919219,131718.750000,6350.000000,5953.125000,0.0,0.0,1454.539551,7168.346250,6817.172344,17255.559844,18101.079844,18101.079844,0.010417,0.010417
100021,2594025,0.000000,675000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100023,1499902,0.000000,135000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [51]:
# getting columns that will be groupbied and summed
credit_sum = credit[list(credit.columns)[15:20]].merge(credit_text, left_index=True, right_index=True)
credit_sum[['SK_ID_CURR','SK_ID_PREV']] = credit[['SK_ID_CURR','SK_ID_PREV']]
credit_sum.head()

,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Approved,SK_ID_CURR,SK_ID_PREV
0,0.0,1,0.0,1.0,35.0,1,0,0,0,0,0,0,378907,2562384
1,1.0,1,0.0,0.0,69.0,1,0,0,0,0,0,0,363914,2582071
2,0.0,0,0.0,0.0,30.0,1,0,0,0,0,0,0,371185,1740877
3,1.0,1,0.0,0.0,10.0,1,0,0,0,0,0,0,337855,1389973
4,0.0,1,0.0,1.0,101.0,1,0,0,0,0,0,0,126868,1891521


In [52]:
# groupbying and taking the sum 
credit_2 = credit_sum.groupby(['SK_ID_CURR','SK_ID_PREV']).sum()
credit_2.head()

,,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Approved
SK_ID_CURR,SK_ID_PREV,,,,,,,,,,,,
100006,1489396,0.0,0,0.0,0.0,0.0,6,0,0,0,0,0,0
100011,1843384,4.0,4,0.0,0.0,1881.0,74,0,0,0,0,0,0
100013,2038692,23.0,23,0.0,0.0,1666.0,96,0,0,0,0,0,0
100021,2594025,0.0,0,0.0,0.0,0.0,7,10,0,0,0,0,0
100023,1499902,0.0,0,0.0,0.0,0.0,8,0,0,0,0,0,0


In [53]:
# getting the max of the days 
credit_3 = credit[['SK_ID_CURR','SK_ID_PREV','SK_DPD','SK_DPD_DEF']].groupby(['SK_ID_CURR','SK_ID_PREV']).max()
credit_3.head()

,,SK_DPD,SK_DPD_DEF
SK_ID_CURR,SK_ID_PREV,,
100006,1489396,0,0
100011,1843384,0,0
100013,2038692,1,1
100021,2594025,0,0
100023,1499902,0,0


In [54]:
# merging the two sub dataframe of credit dataframe 
credit_res = credit_1.merge(credit_2,left_index=True,right_index=True)
credit_res = credit_res.merge(credit_3,left_index=True,right_index=True,suffixes=('_mean','_max')).reset_index().fillna(0)
credit_res.head()

,SK_ID_CURR,SK_ID_PREV,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Approved,SK_DPD_max,SK_DPD_DEF_max
0,100006,1489396,0.000000,270000.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,6,0,0,0,0,0,0,0,0
1,100011,1843384,54482.111149,164189.189189,2432.432432,2432.432432,0.0,0.0,3956.221849,4843.064189,...,1881.0,74,0,0,0,0,0,0,0,0
2,100013,2038692,18159.919219,131718.750000,6350.000000,5953.125000,0.0,0.0,1454.539551,7168.346250,...,1666.0,96,0,0,0,0,0,0,1,1
3,100021,2594025,0.000000,675000.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,7,10,0,0,0,0,0,0,0
4,100023,1499902,0.000000,135000.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,8,0,0,0,0,0,0,0,0


In [56]:
# prev and credit merging 
prev_cre = prev_app_temp.merge(credit_res,on=['SK_ID_PREV','SK_ID_CURR'],how='left',suffixes=('','_credit'))
prev_cre.head()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,NFLAG_INSURED_ON_APPROVAL,CNT_PAYMENT,DAYS_TERMINATION,MISSING_DOWN_PAYMENT,...,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Refused_credit,NAME_CONTRACT_STATUS_Approved_credit,SK_DPD_max,SK_DPD_DEF_max
0,2030495,271877,1730.430,17145.0,17145.0,0.0,0.0,12.0,-37.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2802425,108129,25188.615,607500.0,679671.0,0.0,1.0,36.0,365243.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2523466,122040,15060.735,112500.0,136444.5,0.0,1.0,12.0,365243.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2819243,176158,47041.335,450000.0,470790.0,0.0,1.0,12.0,-177.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1784265,202054,31924.395,337500.0,404055.0,0.0,0.0,24.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
prev_cre.shape

(1670214, 135)

## Installments Payments

In [58]:
install = pd.read_csv('data/installments_payments.csv')
install.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [59]:
install.shape

(13605401, 8)

In [60]:
install['DAYS_ENTRY_PAYMENT'].describe()

count    1.360250e+07
mean    -1.051114e+03
std      8.005859e+02
min     -4.921000e+03
25%     -1.662000e+03
50%     -8.270000e+02
75%     -3.700000e+02
max     -1.000000e+00
Name: DAYS_ENTRY_PAYMENT, dtype: float64

In [61]:
# fillna installment day difference 
install_temp = install[['SK_ID_PREV','SK_ID_CURR','AMT_INSTALMENT','AMT_PAYMENT']]
install_temp['DAY_DIFF'] = install['DAYS_INSTALMENT'].fillna(0) - install['DAYS_ENTRY_PAYMENT'].fillna(0)
install_temp.head()

/var/folders/w1/vc3f0wy917q9_qnbxvz9gvfh0000gn/T/ipykernel_37797/4094312571.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  install_temp['DAY_DIFF'] = install['DAYS_INSTALMENT'].fillna(0) - install['DAYS_ENTRY_PAYMENT'].fillna(0)


,SK_ID_PREV,SK_ID_CURR,AMT_INSTALMENT,AMT_PAYMENT,DAY_DIFF
0,1054186,161674,6948.360,6948.360,7.0
1,1330831,151639,1716.525,1716.525,0.0
2,2085231,193053,25425.000,25425.000,0.0
3,2452527,199697,24350.130,24350.130,8.0
4,2714724,167756,2165.040,2160.585,-17.0


In [62]:
# separate groupby in install
install_1 = install_temp.groupby(['SK_ID_PREV','SK_ID_CURR']).agg({'DAY_DIFF':'mean','AMT_INSTALMENT':'sum',
                                                                   'AMT_PAYMENT':'sum'})
install_1.head()

,,DAY_DIFF,AMT_INSTALMENT,AMT_PAYMENT
SK_ID_PREV,SK_ID_CURR,,,
1000001,158271,16.000000,68443.425,68443.425
1000002,101962,19.750000,37235.565,37235.565
1000003,252457,15.333333,14854.050,14854.050
1000004,260094,26.714286,33523.155,33523.155
1000005,176456,8.454545,161735.310,147021.705


In [63]:
install_1['PAY_INSTALL_PROP'] = install_1['AMT_PAYMENT'].fillna(0) / install_1['AMT_INSTALMENT'].fillna(0)
install_1 = install_1.drop(columns=['AMT_PAYMENT','AMT_INSTALMENT']).reset_index()

In [64]:
#merging all prev records
prev_app = prev_cre.merge(install_1,on=['SK_ID_PREV','SK_ID_CURR'],how='left',suffixes=('','_install'))
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,NFLAG_INSURED_ON_APPROVAL,CNT_PAYMENT,DAYS_TERMINATION,MISSING_DOWN_PAYMENT,...,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Refused_credit,NAME_CONTRACT_STATUS_Approved_credit,SK_DPD_max,SK_DPD_DEF_max,DAY_DIFF,PAY_INSTALL_PROP
0,2030495,271877,1730.430,17145.0,17145.0,0.0,0.0,12.0,-37.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
1,2802425,108129,25188.615,607500.0,679671.0,0.0,1.0,36.0,365243.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.200000,1.0
2,2523466,122040,15060.735,112500.0,136444.5,0.0,1.0,12.0,365243.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.222222,1.0
3,2819243,176158,47041.335,450000.0,470790.0,0.0,1.0,12.0,-177.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.090909,1.0
4,1784265,202054,31924.395,337500.0,404055.0,0.0,0.0,24.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
prev_app.shape

(1670214, 137)

In [66]:
prev_app.fillna(0, inplace=True)
prev_app = prev_app.drop(columns = 'SK_ID_PREV')
prev_app.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,NFLAG_INSURED_ON_APPROVAL,CNT_PAYMENT,DAYS_TERMINATION,MISSING_DOWN_PAYMENT,NAME_CASH_LOAN_PURPOSE_XNA,...,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Refused_credit,NAME_CONTRACT_STATUS_Approved_credit,SK_DPD_max,SK_DPD_DEF_max,DAY_DIFF,PAY_INSTALL_PROP
0,271877,1730.430,17145.0,17145.0,0.0,0.0,12.0,-37.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
1,108129,25188.615,607500.0,679671.0,0.0,1.0,36.0,365243.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.200000,1.0
2,122040,15060.735,112500.0,136444.5,0.0,1.0,12.0,365243.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.222222,1.0
3,176158,47041.335,450000.0,470790.0,0.0,1.0,12.0,-177.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.090909,1.0
4,202054,31924.395,337500.0,404055.0,0.0,0.0,24.0,0.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [68]:
prev_merge = prev_app.drop(columns = 'PAY_INSTALL_PROP').groupby('SK_ID_CURR').sum()
prev_merge['PAY_INSTALL_PROP'] = prev_app[['SK_ID_CURR','PAY_INSTALL_PROP']].groupby('SK_ID_CURR').mean()
curr_pre_df = app_df.merge(prev_merge,left_on='SK_ID_CURR',right_index=True,how='left',suffixes=('', '_prev'))
curr_pre_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,NAME_CONTRACT_STATUS_Completed_prev,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Signed_prev,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Refused_credit,NAME_CONTRACT_STATUS_Approved_credit,SK_DPD_max,SK_DPD_DEF_max,DAY_DIFF,PAY_INSTALL_PROP
0,384575,0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.236842,0.326347
1,214010,0,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146.658748,0.743590
2,142232,0,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.647727,0.272726
3,389171,0,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-29.615385,0.922974
4,283617,0,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.483333,0.400000


In [ ]:
# converting to csv
# prev_app.to_csv('prev_all.csv',index=False)

In [ ]:
#app_bur = pd.read_csv('app_df_bur.csv')
#app_bur

In [ ]:
#pre_total = pd.read_csv('prev_all.csv')
#pre_total

In [ ]:
# saving the total csv 
# curr_pre_df.to_csv('default_df.csv',index=False)

In [262]:
#curr_pre_df.to_csv('merged_final_with_ID.csv',index=False)

## Final Merging

In [69]:
curr_pre_df = curr_pre_df.drop(columns=['SK_ID_CURR'])
curr_pre_df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,NAME_CONTRACT_STATUS_Completed_prev,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Signed_prev,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Refused_credit,NAME_CONTRACT_STATUS_Approved_credit,SK_DPD_max,SK_DPD_DEF_max,DAY_DIFF,PAY_INSTALL_PROP
0,0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.236842,0.326347
1,0,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146.658748,0.743590
2,0,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.647727,0.272726
3,0,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-29.615385,0.922974
4,0,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,234000.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.483333,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,Revolving loans,F,N,Y,1,202500.0,585000.0,29250.0,585000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.901282,0.885556
99996,0,Cash loans,M,N,Y,0,225000.0,562500.0,31540.5,562500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.692424,0.525690
99997,0,Revolving loans,M,Y,N,1,135000.0,180000.0,9000.0,180000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.666667,1.000000
99998,0,Cash loans,F,Y,Y,0,135000.0,254700.0,17149.5,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.500000,0.922380


In [70]:
X = curr_pre_df.drop(columns=['TARGET'])
y = curr_pre_df['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [265]:
#X_train.to_csv('X_train.csv',index=False)
#X_test.to_csv('X_test.csv',index=False)
#y_train.to_csv('y_train.csv',index=False)
#y_test.to_csv('y_test.csv',index=False)

In [73]:
def cleaning(df):
    df['NAME_TYPE_SUITE'] = df['NAME_TYPE_SUITE'].replace({'Other_B':'Other','Other_A':'Other'}).fillna('Other')
#    df['OWN_CAR_AGE'] = df['OWN_CAR_AGE'].fillna(-1)
    df['OCCUPATION_TYPE'] = df['OCCUPATION_TYPE'].fillna('XNA')
    df['LIVING_AVG'] = df[list(df.columns[43:90])].mean(axis=1)
    df['LIVING_MISSING'] = df[list(df.columns[43:90])].isna().sum(axis=1)
#    df = df.drop(columns=list(train_df.columns[42:89]))
    df['SOCIAL_MISSING']=df['DEF_60_CNT_SOCIAL_CIRCLE'].isna().astype('int')
    df['FLAG_DOCUMENT_CNT'] = df[list(df.columns[95:115])].sum(axis =1)
#    df = df.drop(columns=list(df.columns[47:67]))
#    df['CREDIT_TYPE'] = df['CREDIT_TYPE'].fillna('No record')
#    df['STATUS'] = df['STATUS'].fillna('No record')
    return df

In [74]:
train_clean = cleaning(X_train)
test_clean = cleaning(X_test)

/var/folders/w1/vc3f0wy917q9_qnbxvz9gvfh0000gn/T/ipykernel_37797/2244981339.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['LIVING_AVG'] = df[list(df.columns[43:90])].mean(axis=1)


In [75]:
cat_encoder = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
                   selector(dtype_exclude=['float','int']))],
    remainder='passthrough')
X_train_encoded = cat_encoder.fit_transform(X_train)
X_test_encoded = cat_encoder.transform(X_test)

In [76]:
names = [i.replace('cat__','').replace('remainder__','')for i in cat_encoder.get_feature_names_out()]

In [78]:
X_train_encoded = pd.DataFrame(X_train_encoded,columns=names)
X_test_encoded = pd.DataFrame(X_test_encoded,columns=names)
X_train_encoded.head()

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,...,NAME_CONTRACT_STATUS_Refused_credit,NAME_CONTRACT_STATUS_Approved_credit,SK_DPD_max,SK_DPD_DEF_max,DAY_DIFF,PAY_INSTALL_PROP,LIVING_AVG,LIVING_MISSING,SOCIAL_MISSING,FLAG_DOCUMENT_CNT
0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,14.083333,1.342883,7.000000,46.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.213333,0.0,0.0,2.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,15.750000,0.299998,0.000000,46.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,12.545455,0.333333,0.290079,0.0,0.0,1.0
4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20.333333,1.000000,0.000000,46.0,0.0,1.0


In [79]:
#X_train_encoded.to_csv('X_train_cleaned.csv',index=False)
#X_test_encoded.to_csv('X_test_cleaned.csv',index=False)

In [80]:
X_train_encoded = X_train_encoded.replace([np.inf,-np.inf],np.NaN)
X_test_encoded = X_test_encoded.replace([np.inf,-np.inf],np.NaN)

In [305]:
knn_imputer = KNNImputer(n_neighbors=2)
knn_imputer.fit(X_train_encoded)

KNNImputer(n_neighbors=2)

In [306]:
knn_filled_train = knn_imputer.transform(X_train_encoded)
knn_filled_test = knn_imputer.transform(X_test_encoded)

In [82]:
nonrepeating_names = []
for i in names:
    if i in nonrepeating_names:
        nonrepeating_names.append(i + '_prev')
    else:
        nonrepeating_names.append(i)
nonrepeating_names

['NAME_CONTRACT_TYPE_Cash loans',
 'NAME_CONTRACT_TYPE_Revolving loans',
 'CODE_GENDER_F',
 'CODE_GENDER_M',
 'CODE_GENDER_XNA',
 'FLAG_OWN_CAR_N',
 'FLAG_OWN_CAR_Y',
 'FLAG_OWN_REALTY_N',
 'FLAG_OWN_REALTY_Y',
 'NAME_TYPE_SUITE_Children',
 'NAME_TYPE_SUITE_Family',
 'NAME_TYPE_SUITE_Group of people',
 'NAME_TYPE_SUITE_Other',
 'NAME_TYPE_SUITE_Spouse, partner',
 'NAME_TYPE_SUITE_Unaccompanied',
 'NAME_INCOME_TYPE_Businessman',
 'NAME_INCOME_TYPE_Commercial associate',
 'NAME_INCOME_TYPE_Maternity leave',
 'NAME_INCOME_TYPE_Pensioner',
 'NAME_INCOME_TYPE_State servant',
 'NAME_INCOME_TYPE_Student',
 'NAME_INCOME_TYPE_Unemployed',
 'NAME_INCOME_TYPE_Working',
 'NAME_EDUCATION_TYPE_Academic degree',
 'NAME_EDUCATION_TYPE_Higher education',
 'NAME_EDUCATION_TYPE_Incomplete higher',
 'NAME_EDUCATION_TYPE_Lower secondary',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'NAME_FAMILY_STATUS_Civil marriage',
 'NAME_FAMILY_STATUS_Married',
 'NAME_FAMILY_STATUS_Separated',
 'NAME_FAMILY_

In [ ]:
#knn_filled_train = pd.DataFrame(knn_filled_train,columns=nonrepeating_names)
#knn_filled_test = pd.DataFrame(knn_filled_test,columns=nonrepeating_names)

In [339]:
#knn_filled_train.to_csv('X_train_knn.csv',index=False)
#knn_filled_test.to_csv('X_test_knn.csv',index=False)